In [1]:
import csv
import glob
import os
import time
import pandas as pd
import pyodbc

In [2]:
dsn = os.environ['FULL_DATABASE_URL']
conn = pyodbc.connect(dsn)

In [3]:
def get_counts(codes):
    joined_codes = ", ".join(codes)
    sql = f"""
    SELECT count(*)
    FROM medicationissue mi
    JOIN medicationdictionary md ON mi.multilexdrug_id = md.multilexdrug_id
    WHERE mi.consultationdate >= '2019-01-01'
      AND md.dmd_id in ({joined_codes})
    """
    
    for i in range(5):
        try:
            return list(conn.execute(sql.format(", ".join(old_codes))))[0][0]
        except pyodbc.Error as e:
            print(type(e).__name__, e)
            sleep = 10 * 2 ** i
            print(f"Sleeping {sleep} seconds")
            time.sleep(sleep)

In [4]:
counts = {"old": {}, "new": {}}

for path in glob.glob("../local_codelists/*.csv"):
    name = path.split("/")[-1][:-8]
    print(name)

    with open(f"../codelists/{name}.csv") as f:
        rows = list(csv.DictReader(f))

    for header in ["vpid", "dmd_id", "id", "code"]:
        if header in rows[0]:
            break
    else:
        assert False, rows[0].keys()
        
    old_codes = [f"'{r[header]}'" for r in rows]

        
    with open(path) as f:
        rows = list(csv.DictReader(f))

    new_codes = [f"'{r[header]}'" for r in rows]
    
    counts["old"][name] = get_counts(old_codes)
    counts["new"][name] = get_counts(new_codes)

opensafely-ace-inhibitor-medications
opensafely-alpha-adrenoceptor-blocking-drugs
opensafely-angiotensin-ii-receptor-blockers-arbs
opensafely-antidiabetic-drugs
opensafely-antihistamines-oral
opensafely-antiplatelets
opensafely-aspirin
opensafely-asthma-inhaler-salbutamol-medication
opensafely-asthma-inhaler-steroid-medication
opensafely-asthma-oral-prednisolone-medication
opensafely-asthma-related-drug-treatment-codes
opensafely-azithromycin-medication
opensafely-beta-blocker-medications
opensafely-calcium-channel-blockers
opensafely-combination-blood-pressure-medication
opensafely-dmards
opensafely-first-generation-antipsychotics-excluding-long-acting-depots-dmd
opensafely-high-dose-ics-inhalers
opensafely-high-dose-multiple-ingredient-ics-inhalers
opensafely-high-dose-single-ingredient-ics-inhalers
opensafely-ibuprofen-oral
opensafely-indometacin
opensafely-influenza-vaccination
opensafely-insulin-medication
opensafely-laba-ics-combination-inhaler
opensafely-levothyroxine
opensafely

In [5]:
df = pd.DataFrame(counts)
df["delta"] = df["new"] - df["old"]
df["delta %"] = 100 * df["delta"] / df["old"]
df = df.sort_values("delta %", ascending=False)

In [6]:
df

,old,new,delta,delta %
opensafely-levothyroxine,164147,55372271,55208124,33633.343284
opensafely-influenza-vaccination,952,26377,25425,2670.693277
opensafely-sama-medication,62624,387945,325321,519.482946
opensafely-calcium-channel-blockers,13486569,71804648,58318079,432.415976
opensafely-nebulised-asthma-and-copd-medications,267078,1093325,826247,309.365429
opensafely-statin-medication,40626506,125800618,85174112,209.651581
opensafely-asthma-inhaler-salbutamol-medication,12078204,34181467,22103263,183.001239
opensafely-saba-inhaler-medications,12835362,34938625,22103263,172.205996
opensafely-selective-serotonin-reuptake-inhibitors-dmd,41142962,73621383,32478421,78.940405
opensafely-thiazide-type-diuretic-medication,13699033,21697688,7998655,58.388464


In [7]:
print(df.to_markdown(floatfmt=".0f"))

|                                                                              |       old |       new |    delta |   delta % |
|:-----------------------------------------------------------------------------|----------:|----------:|---------:|----------:|
| opensafely-levothyroxine                                                     |    164147 |  55372271 | 55208124 |     33633 |
| opensafely-influenza-vaccination                                             |       952 |     26377 |    25425 |      2671 |
| opensafely-sama-medication                                                   |     62624 |    387945 |   325321 |       519 |
| opensafely-calcium-channel-blockers                                          |  13486569 |  71804648 | 58318079 |       432 |
| opensafely-nebulised-asthma-and-copd-medications                             |    267078 |   1093325 |   826247 |       309 |
| opensafely-statin-medication                                                 |  40626506 | 125800618 |